# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

print(messages[:5])
print(sentiments[:5])

['$FITB great buy at 26.00...ill wait', '@StockTwits $MSFT', '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', '$AMD reveal yourself!']
[4, 3, 4, 3, 2]


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

In [73]:
text = 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG'
#pattern = r'http:[.\S]*'
#pattern = r'@[^ ]+'
pattern = r'\$[^ $]+'
print(re.sub(pattern, '', text))

RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 


### Pre-Processing

In [77]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    
    #print(message[:5])
    text = list(map(lambda x: x.lower(), message))
    #print(text[:5])
    
    # Replace URLs with a space in the message
    pattern = r'http[^ ]+ '
    text = list(map(lambda x: re.sub(pattern, ' ', x), text)) 
    #print(text[:5])
    
    #print(text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    
    pattern = r'\$[^ $]+'
    text = list(map(lambda x: re.sub(pattern, ' ', x), text)) 
    #print(text[:5])
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    
    pattern = r'@[^ ]+[ $]'
    text = list(map(lambda x: re.sub(pattern, ' ', x), text)) 
    #print(text[:5])
    
    # Replace everything not a letter with a space
    
    pattern = r'[^a-zA-Z]+'
    text = list(map(lambda x: re.sub(pattern, ' ', x), text)) 
    #print(text[:5])
        
    # Tokenize by splitting the string on whitespace into a list of words
    
    tokens = list(map(lambda x: x.split(), text))
    #print(tokens[:5])

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.

    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [ [wnl.lemmatize(tok) for tok in token if len(tok)>1 ] for token in tokens]
    print(tokens[:5])
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [7]:
# TODO Implement
tokenized = preprocess(messages)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[' great buy at 26.00...ill wait', '@stocktwits $msft', '#staanalystalert for  : jefferies maintains with a rating of hold setting target price at usd 350.00. our own verdict is buy  http://www.stocktargetadvisor.com/toprating', ' i heard there’s a guy who knows someone who thinks somebody knows something - on stocktwits.', ' reveal yourself!']
[' great buy at 26.00...ill wait', ' $msft', '#staanalystalert for  : jefferies maintains with a rating of hold setting target price at usd 350.00. our own verdict is buy  http://www.stocktargetadvisor.com/toprating', ' i heard there’s a guy who knows someone who thinks somebody knows something - on stocktwits.', ' reveal yourself!']
[' great buy at ill wait', ' msft', ' staanalystalert for jefferies maintains with a rating of hold setting target price at usd our own verdict is buy http www stocktargetadvisor com toprating', ' i heard there s a guy who knows someone who thinks somebody knows something on stocktwits ', ' reveal yourself ']
[['gre

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [8]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

entire_corpus = []
for item in tokenized:
    entire_corpus.extend(item)
    
print(entire_corpus[:20])

bow = Counter(entire_corpus)

print(bow)

['great', 'buy', 'at', 'ill', 'wait', 'msft', 'staanalystalert', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our']
Counter({'the': 406477, 'to': 395129, 'amp': 393698, 'http': 370523, 'com': 323692, 'utm': 305403, 'is': 288608, 'for': 281945, 'on': 247685, 'of': 218059, 'and': 214559, 'in': 212939, 'this': 204418, 'www': 203278, 'it': 199803, 'at': 140638, 'will': 129351, 'up': 122874, 'estimize': 119930, 'source': 110930, 'are': 102733, 'stocktwits': 102271, 'metric': 100567, 'stock': 97961, 'be': 97178, 'medium': 95651, 'you': 95189, 'that': 90281, 'content': 88530, 'short': 87121, 'chart': 82529, 'what': 80085, 'buy': 79757, 'today': 76663, 'here': 75590, 'revenue': 73722, 'just': 73629, 'eps': 73384, 'with': 71755, 'down': 69822, 'call': 69271, 'name': 69233, 'not': 68914, 'report': 68707, 'day': 68649, 'from': 65759, 'intro': 65698, 'we': 65021, 'all': 60761, 'if': 59248, 'now': 58359, 'earnings': 57852, 'growth': 57

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [9]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = bow

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 20

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = sorted(freqs.items(), key= lambda x:x[1], reverse=True)[:high_cutoff]
#print(K_most_common)


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
print(len(filtered_words))

print(filtered_words)

[('the', 406477), ('to', 395129), ('amp', 393698), ('http', 370523), ('com', 323692), ('utm', 305403), ('is', 288608), ('for', 281945), ('on', 247685), ('of', 218059), ('and', 214559), ('in', 212939), ('this', 204418), ('www', 203278), ('it', 199803), ('at', 140638), ('will', 129351), ('up', 122874), ('estimize', 119930), ('source', 110930)]
14982
['great', 'buy', 'at', 'ill', 'wait', 'msft', 'staanalystalert', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'usd', 'our', 'own', 'verdict', 'is', 'http', 'www', 'stocktargetadvisor', 'com', 'toprating', 'heard', 'there', 'guy', 'who', 'know', 'someone', 'think', 'somebody', 'something', 'on', 'stocktwits', 'reveal', 'yourself', 'why', 'the', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position', 'bear', 'have', 'reason', 'to', 'pay', 'more', 'attention', 'dividendbot', 'ba', 'ok', 'good', 'we', 're', 'not', 'dropping', 'in', 'over', 'weekend', 'lol', 'daily', 'chart', 'need', 'get',

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [10]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
id = range(len(filtered_words))
vocab = dict(zip(filtered_words,id))
#print(vocab)

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {v:k for k,v in vocab.items()}
#print(id2vocab)

# tokenized with the words not in `filtered_words` removed.
filtered = [ [word for word in listitem if word in filtered_words ] for listitem in tokenized]
print(filtered[:10])

[['great', 'buy', 'at', 'ill', 'wait'], ['msft'], ['staanalystalert', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'is', 'buy', 'http', 'www', 'stocktargetadvisor', 'com', 'toprating'], ['heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits'], ['reveal', 'yourself'], ['why', 'the', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position'], ['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention', 'http', 'dividendbot', 'com', 'ba'], ['ok', 'good', 'we', 're', 'not', 'dropping', 'in', 'price', 'over', 'the', 'weekend', 'lol'], ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above'], ['drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'in', 'month', 'back', 'to', 'if', 'bo', 'then', 'bingo', 'what', 'is', 'the', 'odds']]


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [11]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [12]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19873289373024636

Finally let's convert our tokens into integer ids which we can pass to the network.

In [13]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [201]:
train_on_gpu=torch.cuda.is_available()
import numpy as np


class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        print('vocab_size:',vocab_size, 'embed_size:',embed_size, 'lstm_size:',lstm_size, 'output_size:',output_size)
        
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        self.dropout = nn.Dropout(dropout)
        self.lstm    = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout = dropout)
        self.dense   = nn.Linear(lstm_size, output_size)
        self.logsoftmax = nn.LogSoftmax()
    


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        #print('lstm layers:', self.lstm_layers,'batch size:', batch_size, 'lstm size:',self.lstm_size)
        
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                            weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
            hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                            weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden_state


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        
        hidden_state = (hidden_state[0].contiguous(), hidden_state[1].contiguous())
        
        batch_size = nn_input.size(1)
        
        #print('nn_input shape:', nn_input.shape)
        
        embeds = self.embedding(nn_input)
        
        #print('embeds shape:', embeds.shape)
        
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        
        #print('lstm out shape:', lstm_out.shape)
        
        out   = self.dropout(lstm_out)
        
        out   = self.dense(out)
        
        #print('out shape:', out.shape)
        
        logps = self.logsoftmax(out)
        
        #print('logps shape:', logps.shape)
        
        logps = logps[-1,:,:].view(logps.size(1),logps.size(2))
        
        #print('log ps.shape:', logps.shape)
        
        logps = logps[:batch_size,:]
        
        #print('logps shape:', logps.shape)
        
        return logps, hidden_state

In [195]:
print(train_on_gpu)

True


### View Model

In [202]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2).cuda()
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64).cuda()
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

vocab_size: 14982 embed_size: 10 lstm_size: 6 output_size: 5
tensor([[-1.5931, -1.5816, -1.5917, -1.5887, -1.6394],
        [-1.6406, -1.5930, -1.5392, -1.5743, -1.6210],
        [-1.6093, -1.5933, -1.5931, -1.5943, -1.6260],
        [-1.5565, -1.6421, -1.6808, -1.5658, -1.6813]], device='cuda:0')


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [203]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = np.array([labels[idx] for idx in indices])

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        #print('batch messages:', batch_messages)
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        #print('labels:',labels[ii: ii+len(batch_messages)] )
        #print('type of labels:', type(labels))
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [204]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

train_ratio = 0.8 

index_list = range(len(sentiments))



train_ids = np.array(random.sample(index_list, int(len(sentiments)*train_ratio)))
valid_ids = np.delete(np.array(index_list),train_ids)

train_features = np.array(token_ids)[train_ids]
valid_features = np.array(token_ids)[valid_ids]

train_labels = np.array(sentiments)[train_ids]
valid_labels = np.array(sentiments)[valid_ids]

print(len(train_features))
print(len(valid_features))
print(len(train_labels))
print(len(valid_labels))

print('train features:')
print(train_features[:5])
print(valid_features[:5])
print(valid_labels[:5])


835541
208886
835541
208886
train features:
[[5089, 721, 214, 8905, 816, 4305, 90, 3996, 79, 54, 3122, 2081]
 [4944, 413, 1526, 327, 22, 1896, 25, 198, 4944, 413, 1526]
 [450, 358, 60, 216, 3773]
 [1723, 292, 5319, 258, 214, 3583, 41, 88, 81, 1196, 781, 4320, 902, 9981, 157, 3204, 3777]
 [4165, 54, 816, 801]]
[[5]
 [42, 74, 75, 76, 77, 78, 79, 80, 64, 81, 72, 52, 78, 82, 83, 84, 85, 21, 41, 86]
 [87, 1]
 [135, 136, 137, 104, 138, 139, 140, 1, 46, 141, 142, 143, 144, 145, 146, 12, 147, 148]
 [6, 7, 154, 155, 156, 16, 15, 10, 11, 12, 1, 14, 15, 16, 2, 17, 18, 19, 20, 21, 1, 22, 23, 24, 25, 26]]
[3 0 4 4 4]


In [205]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.).cuda()
hidden = model.init_hidden(64)

text_batch = text_batch.cuda()
logps, hidden = model.forward(text_batch, hidden)

vocab_size: 14983 embed_size: 200 lstm_size: 128 output_size: 5


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


### Training
It's time to train the neural network!

In [206]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

vocab_size: 14983 embed_size: 1024 lstm_size: 512 output_size: 5


TextClassifier(
  (embedding): Embedding(14983, 1024)
  (dropout): Dropout(p=0.2)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dense): Linear(in_features=512, out_features=5, bias=True)
  (logsoftmax): LogSoftmax()
)

In [207]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 1
batch_size = 1000
learning_rate = 0.0001

print_every = 100
clip = 5

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        
        b_size_train= text_batch.shape[1]
        hidden = model.init_hidden(b_size_train)
        
        b_size = text_batch_val.shape[1]
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
      
        
      
        # TODO Implement: Train Model
        
        model.zero_grad()

        output, hidden = model(text_batch, hidden)
        
        #print('output:')
        #print(output.shape)
        #print(output)
        #print('labels:')
        #print(labels.shape)
        #print(labels)
        
        loss = criterion(output,labels)
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(),clip)
        optimizer.step()

        if steps % print_every == 0:
            model.eval()
            val_losses = []
            
            # TODO Implement: Print metrics
            
            for text_batch_val, labels_val in dataloader(
                valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
                val_h = model.init_hidden(batch_size)
            
                b_size = text_batch_val.shape[1]
                
                val_h = (val_h[0][:,:b_size,:].cuda(), val_h[1][:,:b_size,:].cuda())
        
                #print('val_h shape:', val_h[0].shape, val_h[1].shape)
            
                text_batch_val, labels_val = text_batch_val.cuda(), labels_val.cuda()
            
                output_valid, val_h = model(text_batch_val, val_h)
            
                val_loss = criterion(output_valid,labels_val)
                val_losses.append(val_loss.item())
                
                #print(np.mean(val_losses))
            
            print('Epoch: {}/{}'.format(epoch,epochs),
                  'Step:{}...'.format(steps),
                  'Loss:{:.6f}'.format(loss.item()),
                  'Val Loss:{:.6f}'.format(np.mean(val_losses))
                     )
                       
            model.train()

Starting epoch 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0/1 Step:100... Loss:0.515120 Val Loss:0.530052
Epoch: 0/1 Step:200... Loss:0.060194 Val Loss:0.052880
Epoch: 0/1 Step:300... Loss:0.014116 Val Loss:0.010365
Epoch: 0/1 Step:400... Loss:0.005298 Val Loss:0.004826
Epoch: 0/1 Step:500... Loss:0.002917 Val Loss:0.002340
Epoch: 0/1 Step:600... Loss:0.003852 Val Loss:0.002633
Epoch: 0/1 Step:700... Loss:0.001716 Val Loss:0.001424
Epoch: 0/1 Step:800... Loss:0.001331 Val Loss:0.000777


## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [214]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    #print('text:',text)
    #print('vocab keys:',vocab.keys())
    
    tokens = preprocess([text])
    tokens = tokens[0]
    
    #print('tokens:', tokens)
    
    # Filter non-vocab words
    #print('filter non-vocab words')
    tokens = [token for token in tokens if token in list(vocab.keys())]
    #print('tokens:',tokens) 
    # Convert words to ids
    #print('convert words to ids')
    tokens = [vocab[token] for token in tokens]
    #print('tokens:',tokens)    
        
    # Adding a batch dimension
    
    text_input = torch.tensor(tokens).unsqueeze(1).cuda()
    
    #print('text input:')
    #print(text_input.shape)
    #print(text_input)
    
    # Get the NN output
    hidden = model.init_hidden(text_input.size(1))
    hidden = (hidden[0].cuda(), hidden[1].cuda())
    
    #print('hidden size:', hidden[0].shape)
    
    logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred =  np.exp(logps.detach().cpu().numpy())
    
    return pred

In [215]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
#model.to("cpu")
model.cuda()
pred = predict(text, model, vocab)
print('pred:')
print(pred)

[['google', 'is', 'working', 'on', 'self', 'driving', 'car', 'bullish', 'on']]
pred:
[[  1.59241972e-04   7.44460413e-05   8.10325218e-05   5.92696197e-05
    7.97960820e-05]]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Answer: Prediction is putting new data (validation data or test data) into trained network to have the results. Uncertainty comes from several sources, such as mis-labeling

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [216]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [217]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [218]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [221]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

[['ha', 'moved', 'on', 'check', 'out', 'the', 'movement', 'and', 'peer', 'at', 'http', 'dividendbot', 'com', 'jwn']]
[['bt', 'put', 'when', 'cramer', 'touted', 'going', 'to', 'so', 'for', 'profit', 'and', 'bt', 'back', 'yesterday', 'down', 'on', 'up', 'day', 'is', 'good', 'look', 'lower', 'too']]
[['after', 'er']]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


{'symbol': '$AAPL',
 'score': array([[ 0.26981765,  0.2529366 ,  0.24478474,  0.24724899,  0.25411677]], dtype=float32),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.